In [108]:
import re
gene_list2 = [{'patient': 'Bob' ,'gene1':'1', 'time till death1':'5' , 
               'variants1': 'zz:oops:gcta:- 18:3232232:t-a-t:g 18:3342:t-a-t:c', 
              'gene2':'2', 'time till death2':'7',
               'variants2': '22:394343:gcta:- 18:3232232:t-g-t:-- 18:334342:t-a-t:c', 'comment':'blah'}, 
             {'patient': 'Steve' ,'gene1':'4','time till death1':'3' ,
              'variants1': 'xy:s:zzz:- 18:12:***:. 18:32:t-a-t:.', 
              'gene2':'3', 'time till death2':'9', 
              'variants2': '22:294343:gcta:- 25:explosion:t-a-t:23 12:334342:t-a-t:a', 'comment':'goat'},
              {'patient': 'Jenna' ,'gene1':'7','time till death1':'3' ,
              'variants1': 'n/a', 
              'gene2':'3', 'time till death2':'9', 
              'variants2': 'sx:294343:gcta:- 25:893:t-a-t:a 12:334342:t-a-t:a', 'comment':'Not goat'},
              {'patient': 'Samuele' ,'gene1':'2','time till death1':'11' ,
              'variants1': 'n/a', 
              'gene2':'3', 'time till death2':'9', 
              'variants2': 'chr72:117227792TG>A', 'comment':'Not goat'},
             {'patient': 'Matteo' ,'gene1':'2','time till death1':'11' ,
              'variants1': 'n/a', 
              'gene2':'3', 'time till death2':'9', 
              'variants2': 'OOPSOOPS', 'comment':'Not goat'},
             {'patient': 'Caio' ,'gene1':'2','time till death1':'11' ,
              'variants1': '*', 
              'gene2':'3', 'time till death2':'9', 
              'variants2': 'chr7:117227792>A', 'comment':'Not goat'}]

In [109]:
variants_list =[]
disorder_list = [str(item+1) for item in range(14)]
patient_key= [patient['patient'] for patient in gene_list2]
for person in gene_list2:
    temp_list =[]
    for person_key in person.keys():
        if person_key[-2:-1] =='s':
            temp_list.append(person[person_key])
    variants_list.append(temp_list)
    

In [110]:
for item in variants_list:
    print (item)

['zz:oops:gcta:- 18:3232232:t-a-t:g 18:3342:t-a-t:c', '22:394343:gcta:- 18:3232232:t-g-t:-- 18:334342:t-a-t:c']
['xy:s:zzz:- 18:12:***:. 18:32:t-a-t:.', '22:294343:gcta:- 25:explosion:t-a-t:23 12:334342:t-a-t:a']
['n/a', 'sx:294343:gcta:- 25:893:t-a-t:a 12:334342:t-a-t:a']
['n/a', 'chr72:117227792TG>A']
['n/a', 'OOPSOOPS']
['*', 'chr7:117227792>A']


In [111]:
print(patient_key)

['Bob', 'Steve', 'Jenna', 'Samuele', 'Matteo', 'Caio']


In [112]:
chrom_condition =r"\d+|(xy)+"
chrom_condition_number =r"\d+"
pos_condition =r'\d+'
ref_condition =r'[gcat-]+' 
alt_condition = r'[gcat\.-]+'

cpra_format =r'^(\s*\S+:\S+:\S+:\S+\s*,?\s*)+$'
simp_hgvs_format = r'^(\s*\S+:\S+\s*>\S+)+$'
not_app_format= r'[na/]+'

for patient_variant_data in variants_list:
    for variant_sublist in patient_variant_data:
        sing_variant_sublist = variant_sublist.split(" ")
        '''print ('variant:', 
               disorder_list[patient_variant_data.index(variant_sublist)] ,',Variant List:',
               sing_variant_sublist,
               patient_key[variants_list.index(patient_variant_data)])'''
        for variant in sing_variant_sublist:
            if (re.search(cpra_format, variant)):
                variant_comp_list = variant.split(':')
                #print(variant_comp_list, patient_key[variants_list.index(patient_variant_data)] )
                if (re.search(chrom_condition, variant_comp_list[0], re.I)) ==None:
                    print('Chrom_failure', 'for variant: ', disorder_list[patient_variant_data.index(variant_sublist)], variant)
                if (re.search(chrom_condition_number, variant_comp_list[0], re.I)) and (int(variant_comp_list[0]) >22 or int(variant_comp_list[0]) <0):
                    print('Chrom_failure, not valid', 'for variant: ', disorder_list[patient_variant_data.index(variant_sublist)], variant)
                if (re.search(pos_condition, variant_comp_list[1])) ==None:
                    print('Position failure for ', 'patient:', patient_key[variants_list.index(patient_variant_data)], 
                          'disorder:', disorder_list[patient_variant_data.index(variant_sublist)], 'variant:',variant)
                if (re.search(ref_condition, variant_comp_list[2], re.I)) ==None:
                    print('Ref failure','patient:', patient_key[variants_list.index(patient_variant_data)], 
                          'disorder:', disorder_list[patient_variant_data.index(variant_sublist)], 'variant:',variant)
                if (re.search(alt_condition, variant_comp_list[3], re.I)) ==None:
                    print("Alt-failure", patient_key[variants_list.index(patient_variant_data)], 
                          'disorder:', disorder_list[patient_variant_data.index(variant_sublist)], 'variant:',variant)
            elif re.search(simp_hgvs_format,variant, re.I):
                simp_HGVS_sequence = re.split(r'[\W]+', variant,flags = re.I)
                print ('HGVS Simp', patient_key[variants_list.index(patient_variant_data)], simp_HGVS_sequence[0])
                if simp_HGVS_sequence[0][:3]!="chr" or int(simp_HGVS_sequence[0][3:]) >22:
                    print('Simple HGVS format dictates that chr')
                if (re.search(r'[gcat-]+',simp_HGVS_sequence[2], re.I) == None):
                    print('End point must be CATG')
                if (re.search(r'[gcat-]+',simp_HGVS_sequence[1], re.I) == None):
                    print('Missing CATG') 
                
            elif re.search(not_app_format, variant, re.I):
                print ("Not App", patient_key[variants_list.index(patient_variant_data)])
            elif variant =="*":
                print('Boom', patient_key[variants_list.index(patient_variant_data)])
            elif variant !='*': 
                print ('Error total, not in acceptable format', patient_key[variants_list.index(patient_variant_data)], variant)

Chrom_failure for variant:  1 zz:oops:gcta:-
Position failure for  patient: Bob disorder: 1 variant: zz:oops:gcta:-
Position failure for  patient: Steve disorder: 1 variant: xy:s:zzz:-
Ref failure patient: Steve disorder: 1 variant: xy:s:zzz:-
Ref failure patient: Steve disorder: 1 variant: 18:12:***:.
Chrom_failure, not valid for variant:  2 25:explosion:t-a-t:23
Position failure for  patient: Steve disorder: 2 variant: 25:explosion:t-a-t:23
Alt-failure Steve disorder: 2 variant: 25:explosion:t-a-t:23
Not App Jenna
Chrom_failure for variant:  2 sx:294343:gcta:-
Chrom_failure, not valid for variant:  2 25:893:t-a-t:a
Not App Samuele
HGVS Simp Samuele chr72
Simple HGVS format dictates that chr
Not App Matteo
Error total, not in acceptable format Matteo OOPSOOPS
Boom Caio
HGVS Simp Caio chr7
Missing CATG
